# E-Commerce Revenue Leakage & Customer Experience Analytics

## Project Overview
This project analyzes an e-commerce platform to identify revenue leakage and customer friction points across the purchase funnel.  
The analysis focuses on cart abandonment, payment failures, retry behavior, and customer experience.

## What is done here
csv files required for the project is generated using pandas and numpy






In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

np.random.seed(42)


for creating users.csv

In [3]:
n_users = 2000

users = pd.DataFrame({
    "user_id": range(1, n_users + 1),
    "signup_date": pd.to_datetime("2023-01-01") +
                   pd.to_timedelta(np.random.randint(0, 400, n_users), unit="D"),
    "city": np.random.choice(
        ["Bangalore", "Delhi", "Mumbai", "Hyderabad", "Chennai"],
        n_users
    ),
    "device_type": np.random.choice(["mobile", "desktop"], n_users, p=[0.7, 0.3]),
    "is_returning": np.random.choice([True, False], n_users, p=[0.4, 0.6])
})

users.to_csv("users.csv", index=False)


for creating sessions.csv

In [4]:
n_sessions = 5000

sessions = pd.DataFrame({
    "session_id": range(1, n_sessions + 1),
    "user_id": np.random.choice(users["user_id"], n_sessions),
    "session_start": pd.to_datetime("2024-01-01") +
                     pd.to_timedelta(np.random.randint(0, 60, n_sessions), unit="D"),
    "traffic_source": np.random.choice(
        ["organic", "ads", "referral"],
        n_sessions,
        p=[0.5, 0.35, 0.15]
    ),
    "pages_viewed": np.random.randint(1, 15, n_sessions)
})

sessions.to_csv("sessions.csv", index=False)


for creating product_views.csv

In [5]:
n_views = 12000

product_views = pd.DataFrame({
    "view_id": range(1, n_views + 1),
    "session_id": np.random.choice(sessions["session_id"], n_views),
    "product_id": np.random.randint(100, 500, n_views),
    "category": np.random.choice(
        ["Electronics", "Fashion", "Home", "Beauty"],
        n_views
    ),
    "price": np.random.choice(
        [499, 999, 1499, 2499, 3999, 5999],
        n_views
    )
})

product_views.to_csv("product_views.csv", index=False)


for creating cart_items.csv

In [6]:
n_carts = 6000

cart_items = pd.DataFrame({
    "cart_id": range(1, n_carts + 1),
    "user_id": np.random.choice(users["user_id"], n_carts),
    "product_id": np.random.randint(100, 500, n_carts),
    "quantity": np.random.randint(1, 3, n_carts),
})

cart_items["cart_value"] = cart_items["quantity"] * np.random.choice(
    [499, 999, 1499, 2499, 3999],
    n_carts
)

cart_items["added_at"] = (
    pd.to_datetime("2024-01-01") +
    pd.to_timedelta(np.random.randint(0, 60, n_carts), unit="D")
)

cart_items.to_csv("cart_items.csv", index=False)


for creating orders.csv

In [7]:
n_orders = 3500

orders = pd.DataFrame({
    "order_id": range(1, n_orders + 1),
    "user_id": np.random.choice(cart_items["user_id"], n_orders),
    "order_value": np.random.randint(500, 6000, n_orders),
    "order_status": np.random.choice(
        ["success", "failed", "cancelled"],
        n_orders,
        p=[0.75, 0.15, 0.10]
    ),
    "order_time": pd.to_datetime("2024-01-01") +
                  pd.to_timedelta(np.random.randint(0, 60, n_orders), unit="D")
})

orders.to_csv("orders.csv", index=False)


for creating payments.csv

In [8]:
payments = orders.copy()
payments = payments[payments["order_status"] != "cancelled"]

payments = payments.assign(
    payment_id=range(1, len(payments) + 1),
    payment_method=np.random.choice(
        ["UPI", "Card", "Wallet"],
        len(payments),
        p=[0.5, 0.35, 0.15]
    ),
    bank=np.random.choice(
        ["HDFC", "ICICI", "SBI", "Axis"],
        len(payments)
    ),
    payment_status=np.where(
        payments["order_status"] == "success", "success", "failed"
    ),
    failure_reason=np.where(
        payments["order_status"] == "failed",
        np.random.choice(
            ["network", "timeout", "insufficient_funds", "bank_decline"],
            len(payments)
        ),
        None
    ),
    amount=payments["order_value"],
    payment_time=payments["order_time"] + pd.to_timedelta(
        np.random.randint(1, 10, len(payments)), unit="m"
    )
)

payments = payments[
    ["payment_id", "order_id", "payment_method", "bank",
     "payment_status", "failure_reason", "amount", "payment_time"]
]

payments.to_csv("payments.csv", index=False)


for creating payment_attempts.csv

In [9]:
attempts = []

for _, row in payments.iterrows():
    max_attempts = np.random.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    for attempt in range(1, max_attempts + 1):
        attempts.append({
            "payment_id": row["payment_id"],
            "attempt_number": attempt,
            "attempt_status": (
                "success" if attempt == max_attempts and row["payment_status"] == "success"
                else "failed"
            ),
            "attempt_time": row["payment_time"] + timedelta(minutes=attempt)
        })

payment_attempts = pd.DataFrame(attempts)
payment_attempts.insert(0, "attempt_id", range(1, len(payment_attempts) + 1))

payment_attempts.to_csv("payment_attempts.csv", index=False)


for creating support_tickets.csv

In [10]:
tickets = pd.DataFrame({
    "ticket_id": range(1, 1200),
    "user_id": np.random.choice(users["user_id"], 1199),
    "issue_type": np.random.choice(
        ["payment_failed", "refund_delay", "order_not_received"],
        1199
    ),
    "created_at": pd.to_datetime("2024-01-01") +
                  pd.to_timedelta(np.random.randint(0, 60, 1199), unit="D"),
    "related_payment_id": np.random.choice(
        payments["payment_id"],
        1199
    )
})

tickets.to_csv("support_tickets.csv", index=False)
